In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [55]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Importing dataset

In [56]:
data = pd.read_csv('https://raw.githubusercontent.com/dsrscientist/Data-Science-ML-Capstone-Projects/master/Automobile_insurance_fraud.csv')
data.head() # printing first 5 rows.

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,17-10-2014,OH,250/500,1000,1406.91,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,25-01-2015,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,9935 4th Drive,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,27-06-2006,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,21-01-2015,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,06-09-2000,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,22-02-2015,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7121 Francis Lane,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,25-05-1990,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,10-01-2015,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,06-06-2014,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,17-02-2015,Vehicle Theft,?,Minor Damage,None,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


In this project we will be working with some auto insurance data  create predictive model that predicts if an insurance claim is fraudulent or not.
Our target variable is fraud_reported, By looking on Target we can see its class type classification problem.

In [57]:
### Let's check size of data.
data.shape

(1000, 40)

Our dataset size is 1000 row and 40 columns.

In [58]:
### Checking for null values.
data.isnull().sum()

months_as_customer                0
age                               0
policy_number                     0
policy_bind_date                  0
policy_state                      0
policy_csl                        0
policy_deductable                 0
policy_annual_premium             0
umbrella_limit                    0
insured_zip                       0
insured_sex                       0
insured_education_level           0
insured_occupation                0
insured_hobbies                   0
insured_relationship              0
capital-gains                     0
capital-loss                      0
incident_date                     0
incident_type                     0
collision_type                    0
incident_severity                 0
authorities_contacted             0
incident_state                    0
incident_city                     0
incident_location                 0
incident_hour_of_the_day          0
number_of_vehicles_involved       0
property_damage             

 Great there is no null values in our dataset only one column contain null values _C39 which contain only null values so we can remove it.

In [59]:
data.drop('_c39',axis=1,inplace=True)

In [60]:
### Checking basic info about dataset.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

In our dataset there is no null values also policy_annual_premium columns dtype is Float64,
And 17 columns contain Integer data other all are object dtype.

In [61]:
### Selecting all the catagorical columns  in our dataset.
cols = data.select_dtypes(object)
cols.drop(columns=['incident_location','incident_date','policy_bind_date'],axis=1,inplace=True)

In [62]:
### Printing unique values in our catagorical columns.
for column in cols:
    print(data[column].value_counts().unique)


<bound method Series.unique of OH    352
IL    338
IN    310
Name: policy_state, dtype: int64>
<bound method Series.unique of 250/500     351
100/300     349
500/1000    300
Name: policy_csl, dtype: int64>
<bound method Series.unique of FEMALE    537
MALE      463
Name: insured_sex, dtype: int64>
<bound method Series.unique of JD             161
High School    160
Associate      145
MD             144
Masters        143
PhD            125
College        122
Name: insured_education_level, dtype: int64>
<bound method Series.unique of machine-op-inspct    93
prof-specialty       85
tech-support         78
exec-managerial      76
sales                76
craft-repair         74
transport-moving     72
other-service        71
priv-house-serv      71
armed-forces         69
adm-clerical         65
protective-serv      63
handlers-cleaners    54
farming-fishing      53
Name: insured_occupation, dtype: int64>
<bound method Series.unique of reading           64
exercise          57
paintball    

From above unique values we found that we have ?  values in columns (police_report_available, property_damage, & collision_type).Also we can see there is class imbalance problem in our target variable. First deal with ? values.

In [43]:
data.loc[np.where(data['collision_type'] == '?')]

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
1,228,42,342868,27-06-2006,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,21-01-2015,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
4,228,44,367455,06-06-2014,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,17-02-2015,Vehicle Theft,?,Minor Damage,None,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN
13,121,34,626808,26-10-2012,OH,100/300,1000,936.61,0,464652,FEMALE,MD,armed-forces,bungie-jumping,wife,52800,-32800,08-01-2015,Parked Car,?,Minor Damage,None,SC,Springfield,6582 Elm Lane,5,1,NO,1,1,NO,7280,1120,1120,5040,Toyota,Highlander,2010,N,NaN
26,257,43,863236,20-09-1990,IN,100/300,2000,1322.10,0,436984,MALE,High School,prof-specialty,golf,own-child,0,-29200,28-01-2015,Parked Car,?,Minor Damage,Police,PA,Arlington,2533 Elm St,4,1,YES,1,3,YES,9020,1640,820,6560,Toyota,Camry,2005,N,NaN
27,202,34,608513,18-07-2002,IN,100/300,500,848.07,3000000,607730,MALE,JD,exec-managerial,chess,not-in-family,31000,-30200,07-01-2015,Vehicle Theft,?,Minor Damage,None,VA,Northbrook,3790 Andromedia Hwy,5,1,YES,2,1,?,5720,1040,520,4160,Suburu,Forrestor,2003,Y,NaN
37,289,49,933293,03-02-1993,IL,500/1000,2000,1222.48,0,620757,FEMALE,JD,priv-house-serv,golf,unmarried,0,0,18-01-2015,Parked Car,?,Minor Damage,None,WV,Arlington,6676 Tree Lane,16,1,NO,1,1,YES,5330,1230,820,3280,Suburu,Legacy,2001,N,NaN
48,175,34,866931,07-01-2008,IN,500/1000,1000,1123.87,8000000,446326,FEMALE,PhD,protective-serv,dancing,other-relative,0,0,26-02-2015,Vehicle Theft,?,Trivial Damage,Police,NY,Arlington,6619 Flute Ave,5,1,?,2,0,YES,7290,810,810,5670,Volkswagen,Passat,1995,N,NaN
51,91,27,537546,20-08-1994,IL,100/300,2000,1073.83,0,609930,FEMALE,JD,farming-fishing,polo,husband,0,0,17-01-2015,Vehicle Theft,?,Trivial Damage,None,NY,Arlington,2280 4th Ave,4,1,?,1,2,?,7260,1320,660,5280,BMW,M5,2008,N,NaN
52,217,39,394975,02-06-2002,IN,100/300,1000,1530.52,0,603993,MALE,College,armed-forces,basketball,not-in-family,0,0,22-02-2015,Vehicle Theft,?,Minor Damage,None,WV,Northbend,2644 Elm Drive,8,1,?,2,1,YES,4300,430,430,3440,Toyota,Corolla,2000,N,NaN
54,195,39,282195,17-08-2014,OH,250/500,1000,1393.57,0,478423,MALE,PhD,machine-op-inspct,movies,not-in-family,47600,-39600,27-02-2015,Parked Car,?,Minor Damage,Police,VA,Northbend,5821 2nd St,5,1,NO,0,1,YES,2640,480,480,1680,Ford,F150,2009,N,NaN


By looking above details we understood that all the values are for Vehicle Theft or Parked Car which means  there is no collison hence changing ? with None.

In [63]:
data['collision_type'].replace('?','None',inplace=True) ### replacing ? with None

In [65]:
data.loc[np.where(data['property_damage'] == '?')]

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
1,228,42,342868,27-06-2006,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,21-01-2015,Vehicle Theft,None,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
3,256,41,227811,25-05-1990,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,10-01-2015,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
6,137,34,413978,04-06-2000,IN,250/500,1000,1333.35,0,441716,MALE,PhD,prof-specialty,board-games,husband,0,-77000,13-01-2015,Multi-vehicle Collision,Front Collision,Minor Damage,Police,NY,Springfield,5846 Weaver Drive,0,3,?,0,0,?,78650,21450,7150,50050,Nissan,Pathfinder,2012,N
7,165,37,429027,03-02-1990,IL,100/300,1000,1137.03,0,603195,MALE,Associate,tech-support,base-jumping,unmarried,0,0,27-02-2015,Multi-vehicle Collision,Front Collision,Total Loss,Police,VA,Columbus,3525 3rd Hwy,23,3,?,2,2,YES,51590,9380,9380,32830,Audi,A5,2015,N
16,70,26,558938,08-06-2005,OH,500/1000,1000,1199.44,5000000,619884,MALE,College,machine-op-inspct,hiking,own-child,63600,0,22-02-2015,Multi-vehicle Collision,Rear Collision,Major Damage,Other,NY,Riverwood,5074 3rd St,0,3,?,1,2,YES,52110,5790,5790,40530,Nissan,Maxima,2012,N
19,196,39,143972,02-08-1992,IN,500/1000,2000,1475.73,0,477670,FEMALE,High School,handlers-cleaners,camping,own-child,57000,-27300,22-02-2015,Multi-vehicle Collision,Side Collision,Major Damage,Police,VA,Columbus,8101 3rd Ridge,8,3,?,2,0,NO,60400,6040,6040,48320,Nissan,Pathfinder,2014,N
21,217,41,431876,27-11-2005,IL,500/1000,2000,875.15,0,442479,FEMALE,Associate,machine-op-inspct,skydiving,own-child,46700,0,10-02-2015,Multi-vehicle Collision,Side Collision,Total Loss,Police,SC,Arlington,8957 Weaver Drive,15,3,?,1,2,?,37840,0,4730,33110,Accura,RSX,1996,N
23,413,55,115399,08-02-1991,IN,100/300,2000,1268.79,0,453148,MALE,MD,priv-house-serv,chess,own-child,0,-31000,19-01-2015,Single Vehicle Collision,Front Collision,Total Loss,Ambulance,WV,Northbend,5667 4th Drive,15,1,?,2,2,?,98160,8180,16360,73620,Dodge,RAM,2011,Y
36,62,28,200152,09-03-2003,IL,100/300,1000,988.45,0,430141,FEMALE,Masters,protective-serv,camping,unmarried,0,0,09-01-2015,Single Vehicle Collision,Rear Collision,Total Loss,Police,NY,Northbrook,4981 Weaver St,3,1,?,1,1,YES,60200,6020,6020,48160,Suburu,Forrestor,2004,Y
38,431,54,485664,25-11-2002,IN,500/1000,2000,1155.55,0,615901,FEMALE,MD,craft-repair,bungie-jumping,unmarried,65700,0,21-01-2015,Multi-vehicle Collision,Rear Collision,Major Damage,Police,NY,Hillsdale,3930 Embaracadero St,4,3,?,2,0,?,62300,12460,6230,43610,Jeep,Wrangler,2007,N


In [67]:
temp = data.groupby('collision_type')

In [68]:
temp.get_group("None")

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
1,228,42,342868,27-06-2006,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,21-01-2015,Vehicle Theft,None,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
4,228,44,367455,06-06-2014,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,17-02-2015,Vehicle Theft,None,Minor Damage,None,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,N
13,121,34,626808,26-10-2012,OH,100/300,1000,936.61,0,464652,FEMALE,MD,armed-forces,bungie-jumping,wife,52800,-32800,08-01-2015,Parked Car,None,Minor Damage,None,SC,Springfield,6582 Elm Lane,5,1,NO,1,1,NO,7280,1120,1120,5040,Toyota,Highlander,2010,N
26,257,43,863236,20-09-1990,IN,100/300,2000,1322.10,0,436984,MALE,High School,prof-specialty,golf,own-child,0,-29200,28-01-2015,Parked Car,None,Minor Damage,Police,PA,Arlington,2533 Elm St,4,1,YES,1,3,YES,9020,1640,820,6560,Toyota,Camry,2005,N
27,202,34,608513,18-07-2002,IN,100/300,500,848.07,3000000,607730,MALE,JD,exec-managerial,chess,not-in-family,31000,-30200,07-01-2015,Vehicle Theft,None,Minor Damage,None,VA,Northbrook,3790 Andromedia Hwy,5,1,YES,2,1,?,5720,1040,520,4160,Suburu,Forrestor,2003,Y
37,289,49,933293,03-02-1993,IL,500/1000,2000,1222.48,0,620757,FEMALE,JD,priv-house-serv,golf,unmarried,0,0,18-01-2015,Parked Car,None,Minor Damage,None,WV,Arlington,6676 Tree Lane,16,1,NO,1,1,YES,5330,1230,820,3280,Suburu,Legacy,2001,N
48,175,34,866931,07-01-2008,IN,500/1000,1000,1123.87,8000000,446326,FEMALE,PhD,protective-serv,dancing,other-relative,0,0,26-02-2015,Vehicle Theft,None,Trivial Damage,Police,NY,Arlington,6619 Flute Ave,5,1,?,2,0,YES,7290,810,810,5670,Volkswagen,Passat,1995,N
51,91,27,537546,20-08-1994,IL,100/300,2000,1073.83,0,609930,FEMALE,JD,farming-fishing,polo,husband,0,0,17-01-2015,Vehicle Theft,None,Trivial Damage,None,NY,Arlington,2280 4th Ave,4,1,?,1,2,?,7260,1320,660,5280,BMW,M5,2008,N
52,217,39,394975,02-06-2002,IN,100/300,1000,1530.52,0,603993,MALE,College,armed-forces,basketball,not-in-family,0,0,22-02-2015,Vehicle Theft,None,Minor Damage,None,WV,Northbend,2644 Elm Drive,8,1,?,2,1,YES,4300,430,430,3440,Toyota,Corolla,2000,N
54,195,39,282195,17-08-2014,OH,250/500,1000,1393.57,0,478423,MALE,PhD,machine-op-inspct,movies,not-in-family,47600,-39600,27-02-2015,Parked Car,None,Minor Damage,Police,VA,Northbend,5821 2nd St,5,1,NO,0,1,YES,2640,480,480,1680,Ford,F150,2009,N
